In [15]:
import numpy as np
import scipy as sp
import torch
import pandas as pd
import nlp
from transformers import AutoTokenizer, AutoModelForSequenceClassification

import shap

In [16]:
train = pd.read_csv("../data/train.csv").iloc[:2000,]
test = pd.read_csv("../data/test.csv").iloc[:1000,]

In [18]:
tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-imdb")
model = AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-imdb")

In [25]:
#Predict the logits? Predict something, I'm not sure what.
def f(x):
    tv = torch.tensor(
        [
            tokenizer.encode(v, padding="max_length", max_length=512, truncation=True)
            for v in x
        ]
    )
    outputs = model(tv)[0].detach()
    scores = (np.exp(outputs).T / np.exp(outputs).sum(-1)).T
    val = sp.special.logit(scores[:, 1])  # use one vs rest logit units
    return val
    
explainer = shap.Explainer(f, tokenizer)
shap_values = explainer([test.iloc[55].review], fixed_context=1)

  0%|          | 0/498 [00:00<?, ?it/s]


Partition explainer: 2it [10:46, 646.80s/it]                                                                           


In [26]:
shap.plots.text(shap_values)